In [ ]:
# Jake's original attempt to whittle data down in python. We did a lot of data wrangling in SQL
# before getting to this point. 

# Import libraries

import psycopg2
import numpy as np
import pandas as pd
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
np.random.seed(123)

In [ ]:
conn = psycopg2.connect(database='usaspending', user='postgres', password='gtown2019', host='127.0.0.1', port='5432')

In [ ]:
tbl_name = 'consolidated_data_2_filtered'

In [ ]:
df = pd.read_sql_query('SELECT * FROM ' + tbl_name, con=conn)
df = df[:5]

# Remove duplicative columns

del df['modification_number'] # SQL filter = '0'
del df['awarding_agency_code'] # SQL filter = '97'
del df['primary_place_of_performance_country_code'] # SQL filter = 'USA'
del df['performance_based_service_acquisition_code'] # SQL filter = 'Y'
# Codes for column names
del df['awarding_sub_agency_code']
del df['awarding_office_code']
del df['funding_sub_agency_code']
del df['funding_office_code']
del df['recipient_duns']
del df['award_or_idv_flag']
del df['award_type_code']
del df['type_of_contract_pricing_code']
del df['product_or_service_code']
del df['dod_claimant_program_code']
del df['naics_code']
del df['extent_competed_code']
del df['solicitation_procedures_code']
del df['type_of_set_aside']
del df['performance_based_service_acquisition']
del df['multi_year_contract_code']
del df['contracting_officers_determination_of_business_size_code']

# Remove "manually identified" unimportant columns
del df['action_date']
del df['funding_office_name']
del df['multi_year_contract']
del df['solicitation_procedures']

# Remove "future" data elements
del df['recipient_name']
del df['recipient_country_code']
del df['recipient_state_code']
del df['number_of_offers_received']
del df['extent_competed']
del df['contracting_officers_determination_of_business_size']
del df['number_of_employees']
del df['annual_revenue']

df.shape

In [ ]:
# Create conditional column to determine if set aside is YES or NO

def set_aside(c):
    if c['type_of_set_aside_code'] == 'NONE':
        return 0
    else:
        return 1

def contract_value(c):
    if c['base_and_exercised_options_value'] > 0:
        return c['base_and_exercised_options_value']
    elif c['base_and_all_options_value'] > 0:
        return c['base_and_all_options_value']
    elif c['total_dollars_obligated'] > 0:
        return c['total_dollars_obligated']
    elif c['federal_action_obligation'] > 0:
        return c['federal_action_obligation'] 
    else:
        return 0
    
df['set_aside'] = df.apply(set_aside, axis=1)
df['contract_value'] = df.apply(contract_value, axis=1)

del df['type_of_set_aside_code']
del df['base_and_exercised_options_value']
del df['base_and_all_options_value']
del df['total_dollars_obligated']
del df['federal_action_obligation']

df2 = df.dropna()
df2.shape

In [ ]:
df3 = df2
del df3['contract_transaction_unique_key']
df3 = pd.get_dummies(df2)
df3.head()

In [ ]:
# Pearson Correlation
plt.figure(figsize=(12,10))
cor = df3.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
# Correlation with output variable
cor_target = abs(cor['set_aside'])
# Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]
relevant_features